In [1]:
import torch
from fastai import datasets as FA_dataset
from pathlib import Path
import gzip
import pickle
import math

In [2]:
%matplotlib inline

# import data from previous file

In [3]:
from exp.nb_Matmul import MNIST_URL, fname

In [4]:
MNIST_URL

'http://deeplearning.net/data/mnist/mnist.pkl'

In [5]:
fname

PosixPath('/Users/xianli/Desktop/fast/Part2/data/mnist.pkl.gz')

In [6]:
def get_data_tensors(url,dest):
    path = FA_dataset.download_data(url, dest, ext='.gz')
    with gzip.open(path,'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(torch.tensor, (x_train, y_train, x_valid, y_valid))

def normalize(data, mean, std):
    return (data-mean)/std

In [7]:
x_train, y_train, x_valid, y_valid = get_data_tensors(MNIST_URL,fname)

In [8]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [9]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [10]:
x_train = normalize(x_train, train_mean,train_std)
# note: must use training set mean and std
x_valid = normalize(x_valid, train_mean,train_std)

In [11]:
x_train.mean().item(), x_train.std()

(3.061423922190443e-05, tensor(1.))

In [12]:
x_valid.mean(),x_valid.std()

(tensor(-0.0058), tensor(0.9924))

In [13]:
c = y_train.unique().numel() # number of classes
n,m = x_train.shape

# Initialization test

In [14]:
nh = 50 # number of hidden units
fan_in = 784 # input units

### Simple random initialization for a linear layer

In [15]:
w1 = torch.randn(fan_in,nh)/math.sqrt(fan_in)
b1 = torch.zeros(nh)/math.sqrt(nh)

In [16]:
w1.mean(), w1.std()

(tensor(5.0170e-05), tensor(0.0355))

In [17]:
def lin(x,w,b): return x@w+b

In [18]:
out1 = lin(x_valid,w1,b1)

In [19]:
out1.mean(), out1.std()

(tensor(0.0092), tensor(1.0014))

In [20]:
def relu(y): return y.clamp_min(0.)

In [21]:
x2 = relu(lin(x_valid,w1,b1))

In [22]:
# mean and std of the activations are NOT 0 and 1 due to relu cutting off the negative values
x2.mean(), x2.std()

(tensor(0.3992), tensor(0.6016))

### Kaiming initialization for relu linear layer

In [23]:
w1 = torch.randn(fan_in,nh)*math.sqrt(2./fan_in)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,c)*math.sqrt(2./nh)
b2 = torch.zeros(c)

In [24]:
w1.mean(),w1.std()

(tensor(3.2457e-05), tensor(0.0505))

In [25]:
x1 = relu(lin(x_valid,w1,b1))
x1.mean(),x1.std(),x1.shape

(tensor(0.4931), tensor(0.7554), torch.Size([10000, 50]))

In [26]:
x2 =relu(lin(x1,w2,b2))
x2.mean(),x2.std()

(tensor(0.5102), tensor(0.7362))

### torch version of Kaiming initialization

In [27]:
from torch.nn import init

In [28]:
w1 = torch.zeros(fan_in,nh)
init.kaiming_normal_(w1,mode = 'fan_out')

tensor([[ 0.0547, -0.0208,  0.0193,  ...,  0.0275, -0.0175, -0.0357],
        [ 0.0218, -0.0213,  0.0158,  ...,  0.0830,  0.0357,  0.0437],
        [-0.0119, -0.0219,  0.0352,  ...,  0.0522,  0.0225,  0.0090],
        ...,
        [-0.0388,  0.0043, -0.0166,  ...,  0.0049, -0.0482,  0.0159],
        [-0.0433,  0.0127,  0.0765,  ...,  0.1420, -0.0131,  0.0721],
        [-0.0116,  0.0465,  0.0008,  ...,  0.0268,  0.0171, -0.0060]])

In [29]:
??init.kaiming_normal_

In [30]:
w1.mean(), w1.std()

(tensor(-0.0001), tensor(0.0505))

In [31]:
x1 = relu(lin(x_valid,w1,b1))

In [32]:
x1.mean(), x1.std()

(tensor(0.5276), tensor(0.8088))

# Basic model

In [33]:
def fcc(input, *args, fan_out=10):
    n, p = input.shape
    l_sizes = [p]+args[0]
    print(l_sizes)
    w = []
    b = []
    for i in range(1,len(l_sizes)):
            w.append(init.kaiming_normal_(torch.zeros(l_sizes[i-1],l_sizes[i]),mode='fan_out'))
            b.append(torch.zeros(l_sizes[i]))
    w.append(init.kaiming_normal_(torch.zeros(l_sizes[-1],fan_out),mode='fan_out'))
    b.append(torch.zeros(fan_out))
    x = input
    for i in range(len(w)):
        x = relu(lin(x,w[i],b[i]))
    return x

In [34]:
nh=[50,100,75]
out = fcc(x_valid,nh)

[784, 50, 100, 75]


In [35]:
out.shape

torch.Size([10000, 10])

In [36]:
out.mean(),out.std()

(tensor(0.3661), tensor(0.5288))

In [37]:
%time fcc(x_valid,nh)

[784, 50, 100, 75]
CPU times: user 38.8 ms, sys: 2.57 ms, total: 41.3 ms
Wall time: 26.1 ms


tensor([[0.0000, 0.7566, 0.0000,  ..., 2.0485, 0.4644, 0.0684],
        [0.0000, 1.1972, 0.0000,  ..., 1.5445, 0.4188, 0.9135],
        [0.8977, 1.0240, 0.7300,  ..., 0.6599, 0.5144, 0.3513],
        ...,
        [0.0000, 0.0000, 0.8619,  ..., 0.3754, 0.0244, 0.0000],
        [0.3691, 0.8045, 0.0000,  ..., 1.1513, 0.9783, 0.0000],
        [0.6838, 0.5602, 0.0000,  ..., 1.3448, 0.2002, 0.0000]])

# Layers as classes, basic version

In [38]:
class Lin():
    def __init__(self, w, b):
        self.w = w # dim = (p,q)
        self.b = b # dim = (q)
    
    def __call__(self,x):
        self.x = x # dim = (n,p)
        self.out = x@self.w +self.b # dim = (n,q)
        return self.out
    
    def backward(self):
        # important: every grad is scalar loss function wrt the matrix/vector
        self.x.g = self.out.g @ self.w.t() # dim = (n,q) @ (q, p) = (n,p) 
        # w.g involves batch operation --> one more rank
        # work by using broadcasting
        self.w.g = (self.x.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(dim=0)# dim = (n,p,1) * (n,1,q) --> (n,p,q) -> sum(0) --> (q,p)
        self.b.g = self.out.g.sum(0) # dim = (n,q).sum(0) --> (q)
        

In [39]:
class ReLu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g * (self.inp>0.).float()

In [40]:
# loss function
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp # dim = (n,1)
        self.targ = targ.float() # dim = (n)
#         print('MSE: self.inp.shape = ', inp.shape)
#         print('MSE: self.targ.shape = ', targ.shape)
        return (self.targ-inp.squeeze()).pow(2).sum(0)/ inp.shape[0]
    def backward(self):
        self.inp.g = 2* (self.inp.squeeze() - self.targ).unsqueeze(-1)/self.inp.shape[0] # dim = (n,1)

In [41]:
class Model(): # forward pass calculate loss, backward calculate gradient
    def __init__(self, w1,b1,w2,b2):
        self.layers = [Lin(w1,b1), ReLu(), Lin(w2,b2)]
        self.loss = Mse()
    
    def __call__(self, x, targ): # this model will be called as function
        for l in self.layers:
            # the call inputs for each layer is the same. note each l is an instance of the classes
            # __call__ functions will be used
            x = l(x)
        return self.loss(x,targ)
    
    def backward(self):
        self.loss.backward() # this instance has stored the values of x and targ --> each layer will modify self.x or self.w that itself stores
        for l in reversed(self.layers):
            l.backward()

In [42]:
fan_in = 784
nh = 50
c = 1 # only one class because MSE loss can't work for multiclass problem

w1 = torch.randn(fan_in,nh)*math.sqrt(2./fan_in)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,c)*math.sqrt(2./nh)
b2 = torch.zeros(c)

In [43]:
w1.g, b1.g, w2.g, b2.g = [None] * 4 # need to create the property first
model = Model(w1,b1,w2,b2)
%time loss = model(x_train,y_train) # this is one train cycle, no SDG yet
print('MSE loss = ', loss.item())
# model.backward() # automatically change all parameter gradients

CPU times: user 116 ms, sys: 7.45 ms, total: 124 ms
Wall time: 68.9 ms
MSE loss =  25.117624282836914


In [44]:
x_train.shape, y_train.shape

(torch.Size([50000, 784]), torch.Size([50000]))

In [45]:
%time model.backward() # very very slow backprop

CPU times: user 8.02 s, sys: 24.1 s, total: 32.2 s
Wall time: 34 s


### compare to correct results by pytorch autograd

In [46]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

def mse(y_hat, y):
    return (y_hat.squeeze()-y.float()).pow(2).mean()

def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

loss_2 = forward(xt2,y_train)
%time loss_2.backward() # pytorch autograd
print(loss_2)

CPU times: user 350 ms, sys: 87.8 ms, total: 437 ms
Wall time: 157 ms
tensor(25.1176, grad_fn=<MeanBackward0>)


In [47]:
print(w12.grad)
print(w1.g)

tensor([[-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        ...,
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828]])
tensor([[-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        ...,
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828],
        [-0.1630, -0.1589,  0.0414,  ...,  0.0216, -0.1592, -0.0828]])


In [48]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [49]:
torch.allclose(w12.grad, w1.g, rtol = 1e-3)
torch.allclose(w22.grad, w2.g, rtol = 1e-3)
torch.allclose(b12.grad, b1.g, rtol = 1e-3)
torch.allclose(b22.grad, b2.g, rtol = 1e-3)
torch.allclose(xt2.grad, x_train.g, rtol = 1e-3)


True

True

True

True

True

# Refactor as a module (important)

In [50]:
# outsource all the __call__ function to the base module
# make sure to understand this structure
class myModule():
    def __call__(self, *args):
        self.args = args # self.args is a tuple 
        print('mymodule called')
        self.out = self.forward(*args) # overriding method
        print('mymodule finish child forward call')
        return self.out # report loss from here
    
    def forward(self): # this is always overridden
        raise ValueError('Forward not implemented in root module!')
        
    def backward(self):
        self.bwd(self.out, *self.args) # bwd method to be implemented by the child modules 
        # will automatically get the output of the forward pass


In [51]:
class Lin2(myModule):
    def __init__(self, w, b):
        self.w = w # dim = (p,q)
        self.b = b # dim = (q)
        print('Lin2 class initializes')
    
    def forward(self,x):
        print('Lin2 forward called')
        return x@self.w +self.b # dim = (n,q)
    
    def bwd(self, out, inp):
        # important: every grad is scalar loss function wrt the matrix/vector
        inp.g = out.g @ self.w.t() # dim = (n,q) @ (q, p) = (n,p) 
        # replaced by einsum
#         self.w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(dim=0)# main reason for the slowness
        self.w.g = torch.einsum('ki,kj->ij', inp, out.g)# dim = (n,p,1) * (n,1,q) --> (n,p,q) -> sum(0) --> (q,p)
        self.b.g = out.g.sum(0) # dim = (n,q).sum(0) --> (q)

class ReLu(myModule):
    def forward(self, inp):
        print('Call ReLu forward')
        return inp.clamp_min(0.)
    
    def bwd(self,out,inp):
        inp.g = out.g * (inp>0.).float()

# loss function
class Mse(myModule):
    def forward(self, inp, targ):
        print('Call MSE forward')
        return (targ.float()-inp.squeeze()).pow(2).mean()
    def bwd(self, out, inp, targ):
        inp.g = 2* (inp.squeeze() - targ.float()).unsqueeze(-1)/inp.shape[0] # dim = (n,1)

In [52]:
class Model2():
    def __init__(self,w1,b1,w2,b2):
        self.layers= [Lin2(w1,b1), ReLu(), Lin2(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [53]:
fan_in = 784
nh = 50
c = 1 # only one class because MSE loss can't work for multiclass problem

w1 = torch.randn(fan_in,nh)*math.sqrt(2./fan_in)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,c)*math.sqrt(2./nh)
b2 = torch.zeros(c)

In [54]:
model2 = Model2(w1,b1,w2,b2)
model2(x_train, y_train)

Lin2 class initializes
Lin2 class initializes
mymodule called
Lin2 forward called
mymodule finish child forward call
mymodule called
Call ReLu forward
mymodule finish child forward call
mymodule called
Lin2 forward called
mymodule finish child forward call
mymodule called
Call MSE forward
mymodule finish child forward call


tensor(21.9231)

In [55]:
%time model2.backward() # wall time = 30 s if not using einsum

CPU times: user 252 ms, sys: 79 ms, total: 331 ms
Wall time: 242 ms


# Use pytorch module

In [56]:
from torch import nn

In [57]:
class model(nn.Module):
    def __init__(self, dim_in, nh, dim_out):
        super().__init__()
        self.layers = [nn.Linear(dim_in,nh), nn.ReLU(), nn.Linear(nh,dim_out)]
        self.loss = nn.CrossEntropyLoss()
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x,targ)

In [58]:
dim_in = 784
nh=50
dim_out = 10
torch_model = model(dim_in,nh,dim_out)
loss = torch_model(x_train, y_train)
loss

tensor(2.3075, grad_fn=<NllLossBackward>)

In [59]:
%time loss.backward()

CPU times: user 147 ms, sys: 18.4 ms, total: 165 ms
Wall time: 94 ms
